<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_textos_legales_primeros_pasos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploracion Fallos Jurisprudencia SCBA

In [2]:
# Libraries, tools and thanks 
import bs4
import nltk
import numpy
import pandas as pd
import io
import requests
from google.colab import drive
import numpy as np 
import json

In [4]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [5]:
%cd  'drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [6]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json


# Lectura de tablas de datos primarios

In [7]:
pd.set_option('display.max_columns', None)

In [40]:
with open('tbfallos.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbfallos = json.loads(data1)
tbfallos = pd.DataFrame.from_dict(tbfallos)

In [ ]:
tbfallos['textos_fallo'][1]

In [10]:
with open('tbmetdat.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbmetdat = json.loads(data1)
tbmetdat = pd.DataFrame.from_dict(tbmetdat)

In [11]:
tbmetdat[tbmetdat.nro_causa == 122321]

,nro_causa,fres,textos_raw_len,materia,tipo_fallo,tribunal_emisor,caratula,caratula_pub,magistrados_votantes,tribunal_origen,bnro,magistrados,sumario,voces
1,122321,2019-05-15,160,CIVIL Y COMERCIAL,Sentencia Definitiva,SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA),"Figueredo, Rafael s/ Determinación de la capac...","F. ,R. s/ Determinación de la capacidad jurídica",Soria-de Lázzari-Genoud-Kogan,CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...,158.0,Soria - de Lázzari - Genoud - Kogan.,\nSentencia-Fundamentación. Rendición de cuen...,"RENDICION DE CUENTAS,APROBACION,SENTENCIA,FUND..."


In [12]:
tbmetdat.loc[1]

nro_causa                                                          122321
fres                                                           2019-05-15
textos_raw_len                                                        160
materia                                                 CIVIL Y COMERCIAL
tipo_fallo                                           Sentencia Definitiva
tribunal_emisor          SUPREMA CORTE DE JUSTICIA DE LA PROVINCIA (SCBA)
caratula                Figueredo, Rafael s/ Determinación de la capac...
caratula_pub             F. ,R. s/ Determinación de la capacidad jurídica
magistrados_votantes                        Soria-de Lázzari-Genoud-Kogan
tribunal_origen         CAMARA DE APELACION EN LO CIVIL Y COMERCIAL SA...
bnro                                                                  158
magistrados                          Soria - de Lázzari - Genoud - Kogan.
sumario                  \nSentencia-Fundamentación. Rendición de cuen...
voces                   RENDICION DE C

In [13]:
tbmetdat.loc[1].sumario

' \nSentencia-Fundamentación. Rendición de cuentas-Aprobación. \nLa Suprema Corte de Justicia de la Provincia de Buenos Aires revocó la sentencia que tubo por \naprobada  la rendición  de  cuentas  presentada  por  el  Curador  Oficial  en  el  marco  del  proceso de \ndeterminación de la capacidad, considerando la falta de fundamentación suficiente del fallo y de \nadecuación a los hechos probados en la causa. '

In [14]:
with open('tbdoctrina.json', 'r') as myfile:
    data1=myfile.read()
# parse file
tbdoctrina = json.loads(data1)
tbdoctrina = pd.DataFrame.from_dict(tbdoctrina)

In [15]:
tbdoctrina.head()

,nro_causa,doctrina_nro,doctrina
0,122514,1,El recurso de inaplicabilidad de ley no consti...
1,122514,2,Los juicios de divorcio en trámite al momento ...
2,122514,3,Cuando en la causa sobre divorcio las partes n...
3,122514,4,El artículo 7 del nuevo Código Civil y Comerci...
4,122514,5,La aplicación inmediata de las normas se disti...


In [16]:
tbdoctrina[tbdoctrina.nro_causa == 122321].doctrina

10    Debe ser revocado el fallo que, como único sus...
Name: doctrina, dtype: object

# Preprocesamiento de texto

## Tokenización de sentencias

### 1 Spacy


In [ ]:
# https://spacy.io/usage/spacy-101#whats-spacy
# https://spacy.io/models/es muy iportante
# https://github.com/relopezbriega/relopezbriega.github.io/blob/master/downloads/NLP.ipynb
# Also for english
# https://github.com/GerardoX1/spaCy/blob/master/Spacy%20primeros%20pasos.ipynb o en medium 
# https://medium.com/datos-y-ciencia/comenzando-con-spacy-para-procesamiento-de-lenguaje-natural-e8cf24a18a5a 
# Instalation
# https://stackoverflow.com/questions/49259404/how-to-install-models-download-packages-on-google-colab?rq=1


In [56]:
import spacy.cli
spacy.cli.download("es_core_news_md")
import es_core_news_md
nlp = es_core_news_md.load()

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [57]:
text = tbfallos['textos_fallo'][1]

In [58]:
# Agrego Regla para Segmentar (tokenizar) oraciones
def set_custom_Sentence_end_points(doc):
    for token in doc[:-1]:
        if token.text == '#':
            doc[token.i+1].is_sent_start = True
    return doc

In [59]:
nlp.add_pipe(set_custom_Sentence_end_points, before='parser')

In [60]:
doc=nlp(text)
print(doc)

#TEXTO COMPLETO#A C U E R D O#En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".#A N T E C E D E N T E S#La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).#Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).#Oído el señor Procurador General, dictada la providencia de autos y e

In [ ]:
#Para tokenizar oraciones.
for num,oracion in enumerate(doc.sents):
    print(f'{num}: {oracion}')

# NOT EFFECTIVE

### 2 Script Had-Hoc

Modelo:

https://stackoverflow.com/questions/4576077/how-can-i-split-a-text-into-sentences

Eficacia: Alta

In [92]:
import re
#alphabets= "([A-Za-z])"
#prefixes = "(Sr|Sra|Srta|Dr|Dra)[.]"
#suffixes = "(SA|Ltd)"
#starters = "(Sr|Sra|Srta|Dr|El|Ella|Eso|Ellos|Sus|Nuestros|Nosotros|Pero|Sin embargo|Entonces|Esto|Por)"
#acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
#websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):    
    #text = " " + text + "  "
    #text = text.replace("\n"," ")
    #text = text.replace("#"," ")
    #text = re.sub(prefixes,"\\1<prd>",text)
    #text = re.sub(websites,"<prd>\\1",text)
    #if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    #text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    #text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    #text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    #text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    #text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    #text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    #if "”" in text: text = text.replace(".”","”.")
    #if "\"" in text: text = text.replace(".\"","\".")
    #if "!" in text: text = text.replace("!\"","\"!")
    #if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace("#","<stop>")
    #text = text.replace(".",".<stop>")
    #text = text.replace("?","?<stop>")
    #text = text.replace("!","!<stop>")
    #text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [93]:
split_into_sentences(text)

['',
 'TEXTO COMPLETO',
 'A C U E R D O',
 'En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".',
 'A N T E C E D E N T E S',
 'La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).',
 'Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).',
 'Oído el señor Procurador General, dicta

In [86]:
text_split_122321 =  split_into_sentences(text)

In [77]:
# Para hacer una lista de tokens
lista_palabras=[palabra.text.replace('#',' ') for palabra in doc]
print(lista_palabras)

# EFFECTIVE

[' ', 'TEXTO', 'COMPLETO A', 'C', 'U', 'E', 'R', 'D', 'O En', 'la', 'ciudad', 'de', 'La', 'Plata', ',', 'a', '15', 'de', 'mayo', 'de', '2019', ',', 'habiéndose', 'establecido', ',', 'de', 'conformidad', 'con', 'lo', 'dispuesto', 'en', 'el', 'Acuerdo', '2078', ',', 'que', 'deberá', 'observarse', 'el', 'siguiente', 'orden', 'de', 'votación', ':', 'doctores', 'Soria', ',', 'de', 'Lázzari', ',', 'Genoud', ',', 'Kogan', ',', 'se', 'reúnen', 'los', 'señores', 'Jueces', 'de', 'la', 'Suprema', 'Corte', 'de', 'Justicia', 'en', 'acuerdo', 'ordinario', 'para', 'pronunciar', 'sentencia', 'definitiva', 'en', 'la', 'causa', 'C.', '122.321', ',', '"', 'F.', ',', 'R', '..', 'Determinación', 'de', 'la', 'capacidad', 'jurídica". A', 'N', 'T', 'E', 'C', 'E', 'D', 'E', 'N', 'T', 'E', 'S La', 'Sala', 'I', 'de', 'la', 'Cámara', 'de', 'Apelación', 'en', 'lo', 'Civil', 'y', 'Comercial', 'de', 'San', 'Isidro', 'confirmó', 'el', 'fallo', 'de', 'primera', 'instancia', 'que', ',', 'en', 'los', 'autos', 'sobre', '

## Etiquetado partes del discurso

Una vez realizado tu pipeline y procesada la informacion con los elementos ya tokenizados puedes ver sus atributos instancias que te ayudaran a definir ciertos parametros como el tipo de elemento en el lenguaje, la relación con otros tokens entre muchas otras cualidades con las que cuenta el token, a continuación se muestran las mas tipicas para mas información revisar https://spacy.io/api/token#attributes

- pos_ = tipo de palabra por ejemplo sustantivo, verbo, preposicion   
- tag_ = revision minuciosa de la cualidad anterior espesificando parametros dentro de la categoria   
- dep_ = relacion de dependencia sintactica *

In [84]:
for palabra in doc[100:120]:
    print((palabra.text,palabra.pos_,palabra.tag_,palabra.dep_))

# EFFECTIVE (awasome !)

('I', 'CONJ', 'CCONJ___', 'flat')
('de', 'ADP', 'ADP__AdpType=Prep', 'case')
('la', 'DET', 'DET__Gender=Fem|Number=Sing|PronType=Art', 'det')
('Cámara', 'PROPN', 'PROPN___', 'flat')
('de', 'ADP', 'ADP__AdpType=Prep', 'case')
('Apelación', 'PROPN', 'PROPN___', 'flat')
('en', 'ADP', 'ADP__AdpType=Prep', 'case')
('lo', 'DET', 'DET__Definite=Def|Number=Sing|PronType=Art', 'det')
('Civil', 'PROPN', 'PROPN___', 'nmod')
('y', 'CONJ', 'CCONJ___', 'cc')
('Comercial', 'PROPN', 'PROPN___', 'conj')
('de', 'ADP', 'ADP__AdpType=Prep', 'case')
('San', 'PROPN', 'PROPN___', 'flat')
('Isidro', 'PROPN', 'PROPN___', 'flat')
('confirmó', 'VERB', 'VERB__Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin', 'ROOT')
('el', 'DET', 'DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art', 'det')
('fallo', 'NOUN', 'NOUN__Gender=Masc|Number=Sing', 'obj')
('de', 'ADP', 'ADP__AdpType=Prep', 'case')
('primera', 'ADJ', 'ADJ__Gender=Fem|Number=Sing|NumType=Ord', 'amod')
('instancia', 'NOUN', 'NOUN__Gender=Fem|Number

In [85]:
# Con el siguiente metodo nos devolvera la explicacion de un termino para mas elementos y entender mejor 
# revisar https://github.com/explosion/spaCy/blob/master/spacy/glossary.py
spacy.explain('amod')

'adjectival modifier'

## Dependencia Sintáctiva

El visualizador de dependencias dep muestra etiquetas de parte del discurso y dependencias sintácticas. se coloca el texto a analizar y con el displacy.render podemos observar de forma grafica como se relacionan estos elementos de forma grafica más informacion en https://spacy.io/api/top-level#displacy_options

In [95]:
split_into_sentences(text)

['',
 'TEXTO COMPLETO',
 'A C U E R D O',
 'En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".',
 'A N T E C E D E N T E S',
 'La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).',
 'Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).',
 'Oído el señor Procurador General, dicta

In [88]:
text_split_122321[13]

'La Asesora de Incapaces impugnó la rendición de cuentas efectuada, esgrimiendo que de la documentación acompañada junto a la misma surgían dos facturas de la acompañante terapéutica -Daniela Elizabeth Ferreyra- en concepto de honorarios por trámites de afiliación al Programa Federal de Salud (PROFE-Salud), emitidas con fechas 11 de noviembre de 2015 y 15 de enero de 2016 por las sumas de $720 y $800, respectivamente (v. fs. 566).'

In [89]:
from spacy import displacy
ex=nlp(text_split_122321[13])
displacy.render(ex,style='dep',jupyter=True)

# EFFECTIVE

# Reconocimiento de entidades 1

Realizamos el mismo analisis con un texto mas grande y el resultado fue el siguiente (para mejorar la visualizacion de los datos usaremos un stilo diferente):

In [96]:
displacy.render(doc,style='ent',jupyter=True)

# EFFECTIVE (and amazing!)

# Tools

In [16]:
tbfallos['textos_fallo'][1]

'#TEXTO COMPLETO#A C U E R D O#En la ciudad de La Plata, a 15 de mayo de 2019, habiéndose establecido, de conformidad con lo dispuesto en el Acuerdo 2078, que deberá observarse el siguiente orden de votación: doctores Soria, de Lázzari, Genoud, Kogan, se reúnen los señores Jueces de la Suprema Corte de Justicia en acuerdo ordinario para pronunciar sentencia definitiva en la causa C. 122.321, "F., R.. Determinación de la capacidad jurídica".#A N T E C E D E N T E S#La Sala I de la Cámara de Apelación en lo Civil y Comercial de San Isidro confirmó el fallo de primera instancia que, en los autos sobre determinación de la capacidad jurídica del señor R. F., resolvió aprobar la rendición de cuentas efectuada por la Curaduría Oficial de Alienados (v. fs. 689 y 660, respectivamente).#Se interpuso, por la titular de la Asesoría de Incapaces n° 2 departamental, recurso extraordinario de inaplicabilidad de ley (v. fs. 705/711).#Oído el señor Procurador General, dictada la providencia de autos y 